In [1]:
import pandas as pd
import numpy as np
import statistics

In [2]:
time_list = [date.split(' ')[1].replace('H', ':').replace('M', ':').replace('S', '') for date in np.array(pd.read_csv('src/outputs/result/AOPA1st/ours/result_test.csv', header=None, names=['date', 'pred', 'ans'])['date'])]
aopa1_preds = [pred for pred in np.array(pd.read_csv('src/outputs/result/AOPA1st/ours/result_test.csv', header=None, names=['date', 'pred', 'ans'])['pred'])]
aopa1_anss = [ans for ans in np.array(pd.read_csv('src/outputs/result/AOPA1st/ours/result_test.csv', header=None, names=['date', 'pred', 'ans'])['ans'])]

len(time_list)

26736

In [3]:
class_label = ['Chores', 'Paperwork', 'Cylinder preparation', 'Material preparation', 'Ink roller adjustment', 'Winding preparation (top)', 'Winding preparation (bottom)', 'Creation of sample', 'Ink adjustment', 'Workbench', 'Product check', 'Nonhuman work'
]

In [4]:
def mode_by_time(data):
    outputs = {}
    for d in data:
        time = d[0]
        pred = d[1]
        label = d[2]+1
        try:
            outputs[time]['pred'] += [pred]
            outputs[time]['label'] += [label]
        except:
            outputs[time] = {'pred': [pred], 'label': [label]}
    moded = []
    #for key, item in sorted(outputs.items()):
    for key in time_list:
        try:
            pred = outputs[key]['pred'] #item['pred']
            label = outputs[key]['label'] #item['label']
            pred = statistics.mode(pred)
            label = statistics.mode(label)
            moded += [[key, pred, label]]
        except:
            print(f'"{key}",')
    moded = np.array(moded)
    print(moded.shape)
    return moded

In [5]:
def confusion_matrix(data):
    cm = np.zeros((12, 12), dtype=np.int32)
    for d in data:
        label = int(d[2])
        pred = int(d[1])
        cm[label, pred] += 1
    return cm

In [6]:
def confusion_matrix_from_path(p):
    outputs = np.array(pd.read_csv(p))
    outputs = mode_by_time(outputs)
    cm = confusion_matrix(outputs)
    return cm

In [7]:
def calc_scores(cm):
    corrects = np.diag(cm)
    correct = corrects.sum()
    datanum = cm.sum()

    recalls = corrects / (cm.sum(axis=1) + 1e-20)
    precisions = corrects / (cm.sum(axis=0) + 1e-20)
    f1s = 2 * precisions * recalls / (precisions + recalls + 1e-20)

    acc = correct / datanum
    recall = recalls.mean()
    precision = precisions.mean()
    f1 = f1s.mean()
    scores = {'recalls': recalls, 'precisions': precisions, 'f1s': f1s, 'acc': acc, 'recall': recall, 'precision': precision, 'f1': f1}
    return scores

In [8]:
def show_confusion_matrix(cm):
    # view table of markdown
    print('↓正解      予測→')
    print('|    |', '|'.join([f'  {pred+1:=2}  ' for pred in range(12)]), '|')
    print('|----|', '|'.join(['------' for num in range(12)]), '|')
    for label, row in enumerate(cm):
        print(f'| {label+1:=2} |', '|'.join([f'{num:=5} ' for num in row]), '|')
    
    # view table of tex
    print()
    for label, row in enumerate(cm):
        print(f'    & {label+1:=2} &', '&'.join([f'{num:=5} ' for pred, num in enumerate(row)]), '&    \\ \cline{2-14}')

def show_class_score(scores, score_type='f1s'):
    models = sorted(scores.keys())[::-1]
    score = np.array([scores[model][score_type] for model in models]).T
    df = pd.DataFrame(score, columns=[model.split('/')[0] for model in models], index=[f'{i+1} {cl}' for i, cl in enumerate(class_label)])
    print(df)
    print()

def show_class_scores(scores):
    pd.set_option('display.expand_frame_repr', False)
    print('== SCORES ==')
    print(sorted(scores.keys())[::-1])
    print()
    print('recalls')
    show_class_score(scores, 'recalls')
    print('precisions')
    show_class_score(scores, 'precisions')
    print('f1s')
    show_class_score(scores, 'f1s')
    
def show_describe(scores):
    acc, recall, precision, f1 = scores['acc'], scores['recall'], scores['precision'], scores['f1']
    print('|     | score(%) |')
    print('|-----|----------|')
    print(f'| acc | {acc:.5f} |')
    print(f'|  re | {recall:.6f} |')
    print(f'|  pr | {precision:.6f} |')
    print(f'|  f1 | {f1:.6f} |')


In [9]:
# old_result = pd.read_csv('src/outputs/result/AOPA1st/ours/result_test.csv', header=None, names=['date', 'pred', 'ans'])
# padding_times = [
# "08:32:50",
# "08:55:51",
# "08:55:52",
# "08:55:53",
# "08:55:54",
# "08:55:55",
# "08:55:56",
# "08:55:57",
# "08:55:58",
# "08:55:59",
# "08:56:00",
# "08:56:01",
# "08:56:02",
# "08:56:03",
# "08:56:04",
# "08:56:05",
# "08:56:06",
# "08:56:07",
# "08:56:08",
# "08:56:09",
# "08:56:10",
# "08:56:11",
# "08:56:12",
# "08:56:13",
# "08:56:14",
# "08:56:15",
# "08:56:16",
# "08:56:17",
# "08:56:18",
# "08:56:19",
# "08:56:20",
# "08:56:21",
# "08:56:22",
# "08:56:23",
# "08:56:24",
# "08:56:25",
# "08:56:26",
# "08:56:27",
# "08:56:28",
# "08:56:29",
# "08:56:30",
# "08:56:31",
# "08:56:32",
# "08:56:33",
# "08:56:34",
# "08:56:35",
# "08:56:36",
# "08:56:37",
# "08:56:38",
# "08:56:39",
# "08:56:40",
# "08:56:41",
# "08:56:42",
# "08:56:43",
# "08:56:44",
# "08:56:45",
# "08:56:46",
# "08:56:47",
# "08:56:48",
# "08:56:49",
# "08:56:50",
# "08:56:51",
# "08:56:52",
# "08:56:53",
# "08:56:54",
# "08:56:55",
# "08:56:56",
# "08:56:57",
# "08:56:58",
# "08:56:59",
# "08:57:00",
# "08:57:01",
# "08:57:02",
# "08:57:03",
# "08:57:04",
# "08:57:05",
# "08:57:06",
# "08:57:07",
# "08:57:08",
# "08:57:09",
# "08:57:10",
# "08:57:11",
# "08:57:12",
# "08:57:13",
# "08:57:14",
# "08:57:15",
# "08:57:16",
# "08:57:17",
# "08:57:18",
# "08:57:19",
# "08:57:20",
# "08:57:21",
# "08:57:22",
# "08:57:23",
# "08:57:24",
# "08:57:25",
# "08:57:26",
# "08:57:27",
# "08:57:28",
# "08:57:29",
# "08:57:30",
# "08:57:31",
# "08:57:32",
# "09:35:53",
# "09:35:56",
# "10:35:58",
# "11:08:50",
# "11:08:51",
# "11:08:52",
# "11:08:53",
# "11:08:54",
# "11:08:55",
# "11:08:56",
# "11:08:57",
# "11:08:58",
# "11:08:59",
# "11:09:00",
# "11:09:01",
# "11:09:02",
# "11:09:03",
# "11:09:04",
# "11:09:05",
# "11:09:06",
# "11:09:07",
# "11:09:08",
# "11:09:09",
# "11:09:10",
# "11:09:11",
# "11:09:12",
# "11:09:13",
# "11:09:14",
# "11:09:15",
# "11:09:16",
# "11:09:17",
# "11:09:18",
# "11:09:19",
# "11:09:20",
# "11:09:21",
# "11:09:22",
# "11:09:23",
# "11:09:24",
# "11:09:25",
# "11:09:26",
# "11:09:27",
# "11:09:28",
# "11:09:29",
# "11:09:30",
# "11:09:31",
# "11:09:32",
# "11:09:33",
# "11:09:34",
# "11:09:35",
# "11:09:36",
# "11:09:37",
# "11:09:38",
# "11:09:39",
# "11:09:40",
# "11:09:41",
# "11:09:42",
# "11:09:43",
# "11:09:44",
# "11:09:45",
# "11:09:46",
# "11:09:47",
# "11:09:48",
# "11:09:49",
# "11:09:50",
# "11:09:51",
# "11:09:52",
# "11:09:53",
# "11:09:54",
# "11:09:55",
# "11:09:56",
# "11:09:57",
# "11:09:58",
# "11:09:59",
# "11:10:00",
# "11:10:01",
# "11:10:02",
# "11:10:03",
# "11:10:04",
# "11:10:05",
# "11:10:06",
# "11:10:07",
# "11:10:08",
# "11:10:09",
# "11:10:10",
# "11:10:11",
# "11:10:12",
# "11:10:13",
# "11:10:14",
# "11:10:15",
# "11:10:16",
# "11:10:17",
# "11:10:18",
# "11:10:19",
# "11:10:20",
# "11:10:21",
# "11:10:22",
# "11:10:23",
# "11:10:24",
# "11:10:25",
# "11:10:26",
# "11:10:27",
# "11:10:28",
# "11:10:29",
# "11:10:30",
# "11:10:31",
# "11:10:32",
# "11:10:33",
# "11:10:34",
# "11:10:35",
# "11:10:36",
# "11:10:37",
# "11:10:38",
# "11:10:39",
# "11:10:40",
# "11:10:41",
# "11:10:42",
# "11:10:43",
# "11:10:44",
# "11:10:45",
# "11:10:46",
# "11:10:47",
# "11:10:48",
# "11:10:49",
# "11:10:50",
# "11:10:51",
# "11:10:52",
# "11:10:53",
# "11:10:54",
# "11:10:55",
# "11:10:56",
# "11:10:57",
# "11:10:58",
# "11:10:59",
# "11:11:00",
# "11:11:01",
# "11:11:02",
# "11:11:03",
# "11:11:04",
# "11:11:05",
# "11:11:06",
# "11:11:07",
# "11:11:08",
# "11:11:09",
# "11:11:10",
# "11:11:11",
# "11:11:12",
# "11:11:13",
# "11:11:14",
# "11:11:15",
# "11:11:16",
# "11:11:17",
# "11:11:18",
# "11:11:19",
# "11:11:20",
# "11:11:21",
# "11:11:22",
# "11:11:23",
# "11:11:24",
# "11:11:25",
# "11:11:26",
# "11:11:27",
# "11:11:28",
# "11:11:29",
# "11:11:30",
# "11:11:31",
# "11:11:32",
# "11:11:33",
# "11:11:34",
# "11:11:35",
# "11:11:36",
# "11:11:37",
# "11:11:38",
# "11:11:39",
# "11:11:40",
# "11:11:41",
# "11:11:42",
# "11:11:43",
# "11:11:44",
# "12:16:35",
# "13:02:55",
# ]
# padding_times = [f"15D {t.split(':')[0]}H{t.split(':')[1]}M{t.split(':')[2]}S" for t in padding_times]
# for ans in [old_result[old_result.date==t]['ans'] for t in padding_times]:
#     print(f'{int(ans)},')


In [10]:
from glob import glob

base_path = 'src/outputs/result'
models = ['feat', 'feat-sensor-small']
model = models[1]
checkname = 'lr001b16wDST2C12'
split = 'test'

scores = {}

path = glob(f'{base_path}/{model}/{checkname}/*/{split}.csv')[0]
print(path)
cm = confusion_matrix_from_path(path)
show_confusion_matrix(cm)
scores[model] = calc_scores(cm)
show_class_scores(scores)
show_describe(scores[model])

src/outputs/result/feat-sensor-small/lr001b16wDST2C12/C12_ep20/test.csv
(26736, 3)
↓正解      予測→
|    |    1  |   2  |   3  |   4  |   5  |   6  |   7  |   8  |   9  |  10  |  11  |  12   |
|----| ------|------|------|------|------|------|------|------|------|------|------|------ |
|  1 |   361 |  132 |  323 |  213 |   95 |   68 |   75 |   28 |   53 |  161 |  175 |   42  |
|  2 |    67 | 1424 |  137 |    9 |   16 |    8 |    5 |  151 |    0 |    6 |   52 |    4  |
|  3 |   287 |   94 | 2983 |   25 |   39 |    1 |  283 |    1 |   47 |  154 |   35 |    1  |
|  4 |    17 |    5 |   20 | 2328 |    1 |    0 |   28 |    1 |   38 |    1 |   32 |   72  |
|  5 |     0 |    3 |   20 |    0 | 1081 |   64 |  190 |    6 |   11 |    0 |   52 |    0  |
|  6 |    30 |    2 |    0 |   22 |  140 | 1013 |  117 |  118 |    5 |    1 |   38 |    0  |
|  7 |     1 |    4 |    3 |   23 |   12 |   53 |  731 |   84 |    1 |    0 |    1 |   15  |
|  8 |    69 |  288 |   15 |    3 |   52 |  323 |  329 | 1213 |   3

In [11]:
# AOPA1

aopa1_moded = []
for key, pred, ans in zip(time_list, aopa1_preds, aopa1_anss):
    try:
        aopa1_moded += [[key, pred, ans]]
    except:
        print(f'"{key}",')
aopa1_moded = np.array(aopa1_moded)
aopa1_cm = confusion_matrix(aopa1_moded)
show_confusion_matrix(aopa1_cm)
aopa1_score = {'aopa1': calc_scores(aopa1_cm)}
show_class_scores(aopa1_score)
show_describe(aopa1_score['aopa1'])


↓正解      予測→
|    |    1  |   2  |   3  |   4  |   5  |   6  |   7  |   8  |   9  |  10  |  11  |  12   |
|----| ------|------|------|------|------|------|------|------|------|------|------|------ |
|  1 |   946 |   67 |  161 |   38 |    4 |   23 |   77 |  162 |   41 |   49 |  118 |   40  |
|  2 |   232 | 1227 |  253 |   31 |    6 |   27 |    9 |   37 |    4 |    7 |   38 |    8  |
|  3 |   160 |   12 | 3080 |   30 |    1 |    2 |  443 |   31 |  131 |   47 |    7 |    6  |
|  4 |   117 |   11 |   13 | 2199 |    2 |    2 |    9 |    0 |   56 |    0 |   66 |   68  |
|  5 |   144 |   24 |   77 |    5 |  683 |   61 |  240 |   44 |    0 |    0 |  149 |    0  |
|  6 |    31 |   15 |   48 |    8 |   61 |  793 |  124 |  352 |    4 |    0 |   39 |   11  |
|  7 |    18 |    0 |   10 |   12 |   12 |    3 |  668 |  197 |    0 |    0 |    8 |    0  |
|  8 |   429 |  246 |   28 |    7 |   19 |  427 |  220 | 1193 |   14 |    0 |   94 |   18  |
|  9 |    29 |    0 |    0 |    1 |    0 |    0 |  268 | 

In [12]:
# V.S.
vs_score = scores
vs_score['z_aopa1'] = aopa1_score['aopa1']
show_class_scores(vs_score)

# mix
ours_is_better = [1, 2, 4, 5, 6, 7, 10, 11]
mix_cm = aopa1_cm
mix_cm[ours_is_better] = cm[ours_is_better]
show_confusion_matrix(mix_cm)
vs_score['0_mix'] = calc_scores(mix_cm)
show_class_scores(vs_score)
show_describe(vs_score['0_mix'])

== SCORES ==
['z_aopa1', 'feat-sensor-small']

recalls
                                 z_aopa1  feat-sensor-small
1 Chores                        0.548088           0.209154
2 Paperwork                     0.653007           0.757850
3 Cylinder preparation          0.779747           0.755190
4 Material preparation          0.864727           0.915454
5 Ink roller adjustment         0.478626           0.757533
6 Winding preparation (top)     0.533647           0.681696
7 Winding preparation (bottom)  0.719828           0.787716
8 Creation of sample            0.442672           0.450093
9 Ink adjustment                0.465995           0.331234
10 Workbench                    0.368421           0.517544
11 Product check                0.677482           0.716879
12 Nonhuman work                0.932368           0.970112

precisions
                                 z_aopa1  feat-sensor-small
1 Chores                        0.353909           0.408834
2 Paperwork                     0

In [16]:
from glob import glob

base_path = 'src/outputs/result'
models = ['feat', 'feat-sensor-small']
model = models[1]
checkname = [f'lr001b16wDST2C{num}' for num in range(1, 33)]
split = 'test'

scores = {}

for cn in checkname:
    path = glob(f'{base_path}/{model}/{cn}/*/{split}.csv')[0]
    print(path)
    cm = confusion_matrix_from_path(path)
    show_confusion_matrix(cm)
    scores[model] = calc_scores(cm)
    show_class_scores(scores)
    show_describe(scores[model])

src/outputs/result/feat-sensor-small/lr001b16wDST2C1/C1_ep38/test.csv
(26453, 3)
↓正解      予測→
|    |    1  |   2  |   3  |   4  |   5  |   6  |   7  |   8  |   9  |  10  |  11  |  12   |
|----| ------|------|------|------|------|------|------|------|------|------|------|------ |
|  1 |   613 |  105 |    6 |   43 |    2 |   67 |   34 |   87 |   14 |  540 |  135 |   10  |
|  2 |    77 | 1101 |    1 |    6 |    0 |    4 |    0 |  544 |    0 |  110 |   30 |    5  |
|  3 |   874 |   79 |  987 |   35 |    7 |   55 |  301 |   11 |   77 | 1494 |   27 |    3  |
|  4 |   127 |    2 |    1 | 2101 |    0 |    0 |    5 |   15 |   25 |  137 |   30 |   70  |
|  5 |   200 |    1 |   18 |    0 |  849 |   13 |  187 |   56 |    0 |    3 |   96 |    0  |
|  6 |    94 |    2 |    0 |    0 |   40 |  947 |   41 |  290 |    6 |    8 |   43 |    1  |
|  7 |   211 |    1 |    0 |    2 |    6 |   43 |  532 |  105 |    1 |   10 |   11 |    0  |
|  8 |   166 |  126 |    0 |    3 |   53 |  220 |  533 | 1421 |    0 

In [57]:
models = ['feat', 'feat-mask', 'feat-mask-sensor', 'feat-sensor-small']
param = '/lr001b16wD_loss'
split = '/test'
models = [model+param+split for model in models]

scores = {}
for model in models:
    print(model)
    cm = confusion_matrix_from_path(f'src/outputs/{model}.csv')
    show_confusion_matrix(cm)
    scores[model] = calc_scores(cm)


feat/lr001b16wD_loss/test
(24844, 3)
↓正解      予測→
|    |    1  |   2  |   3  |   4  |   5  |   6  |   7  |   8  |   9  |  10  |  11  |  12   |
|----| ------|------|------|------|------|------|------|------|------|------|------|------ |
|  1 |   602 |   75 |   88 |   36 |   11 |   56 |   46 |   10 |    0 |  275 |   72 |   31  |
|  2 |   136 | 1453 |   16 |    5 |    0 |    8 |    4 |  219 |    0 |  125 |   33 |    1  |
|  3 |   875 |  211 | 2348 |   36 |    9 |   14 |  381 |   18 |   59 |  676 |   83 |    4  |
|  4 |   124 |    4 |    5 | 1700 |    0 |    0 |   16 |    3 |   20 |  106 |   32 |   46  |
|  5 |    80 |    4 |    7 |    0 |  385 |  247 |  332 |   39 |    2 |    0 |  170 |    5  |
|  6 |    91 |   10 |    2 |    0 |   11 |  973 |   60 |  114 |    0 |    5 |   41 |    1  |
|  7 |    30 |    0 |    0 |    1 |    7 |  208 |  474 |   33 |    0 |    3 |   12 |    0  |
|  8 |   275 |  722 |    0 |    3 |    1 |  356 |  312 |  466 |    0 |   10 |   68 |  127  |
|  9 |    37 |    0 

In [29]:
show_class_scores(scores)

== SCORES ==
['feat/lr001b16wD_loss/test', 'feat-sensor-small/lr001b16wD_loss/test', 'feat-mask/lr001b16wD_loss/test', 'feat-mask-sensor/lr001b16wD_loss/test']

recalls
                                    feat  feat-sensor-small  feat-mask  feat-mask-sensor
1 Chores                        0.462366           0.441824   0.237898          0.282502
2 Paperwork                     0.726500           0.509757   0.873134          0.805515
3 Cylinder preparation          0.498091           0.669171   0.730418          0.297339
4 Material preparation          0.826848           0.840352   0.864937          0.784011
5 Ink roller adjustment         0.302911           0.672156   0.536567          0.300919
6 Winding preparation (top)     0.743884           0.678851   0.575854          0.691792
7 Winding preparation (bottom)  0.617188           0.590296   0.476323          0.688645
8 Creation of sample            0.199145           0.468996   0.134042          0.512799
9 Ink adjustment              

In [30]:
for model in sorted(models)[::-1]:
    print(model)
    show_describe(scores[model])

feat/lr001b16wD_loss/test
|     | score(%) |
|-----|----------|
| acc | 0.65247 |
|  re | 0.571298 |
|  pr | 0.562192 |
|  f1 | 0.509847 |
feat-sensor-small/lr001b16wD_loss/test
|     | score(%) |
|-----|----------|
| acc | 0.68801 |
|  re | 0.633624 |
|  pr | 0.596894 |
|  f1 | 0.579643 |
feat-mask/lr001b16wD_loss/test
|     | score(%) |
|-----|----------|
| acc | 0.66036 |
|  re | 0.565001 |
|  pr | 0.541526 |
|  f1 | 0.499136 |
feat-mask-sensor/lr001b16wD_loss/test
|     | score(%) |
|-----|----------|
| acc | 0.63496 |
|  re | 0.587904 |
|  pr | 0.589169 |
|  f1 | 0.530444 |


↓正解      予測→
|    |    1  |   2  |   3  |   4  |   5  |   6  |   7  |   8  |   9  |  10  |  11  |  12   |
|----| ------|------|------|------|------|------|------|------|------|------|------|------ |
|  1 |   669 |  181 |    8 |   54 |   17 |  113 |   35 |   43 |   80 |  237 |  219 |   52  |
|  2 |   151 | 1447 |   10 |    7 |    7 |   15 |    2 |  699 |    0 |   23 |  100 |    2  |
|  3 |  1704 |  347 | 1502 |  175 |   13 |   56 |  614 |   64 |  219 |  714 |  213 |   74  |
|  4 |   345 |   35 |    2 | 1820 |    1 |    6 |    2 |  116 |   10 |   18 |   97 |  113  |
|  5 |   104 |    4 |    1 |    0 |  492 |  484 |  364 |    9 |   33 |    1 |  148 |    1  |
|  6 |    72 |   35 |    0 |    0 |    9 | 1054 |   22 |  186 |    0 |   18 |   73 |    3  |
|  7 |   219 |    8 |    0 |    4 |    0 |  148 |  311 |  202 |    0 |    6 |   13 |   11  |
|  8 |   260 |  394 |    0 |    3 |    7 |  573 |  214 | 1165 |    0 |    8 |  177 |   45  |
|  9 |    50 |    0 |    5 |   67 |    8 |   59 |  214 |    1 |  126 |    0 |  242 |    0  |
| 10 |    78 |   49 |   14 |    3 |    0 |    0 |    0 |    1 |    0 |   86 |    2 |    4  |
| 11 |   130 |    8 |    3 |   78 |   49 |  571 |   22 |   29 |   61 |    3 | 1520 |    0  |
| 12 |   136 |   57 |    0 |   35 |    1 |    6 |    1 |  290 |    5 |    2 |   19 | 6701  |


↓正解      予測→
|    |    1  |   2  |   3  |   4  |   5  |   6  |   7  |   8  |   9  |  10  |  11  |  12   |
|----| ------|------|------|------|------|------|------|------|------|------|------|------ |
|  1 |   284 |    7 |  368 |   92 |    2 |   76 |   86 |  163 |   90 |  353 |  141 |   46  |
|  2 |   103 |    5 |  104 |   14 |    0 |    3 |    4 | 1965 |    8 |  185 |   71 |    1  |
|  3 |   264 |    0 | 3417 |   35 |    0 |   33 |  120 |   53 |  458 | 1107 |  184 |   24  |
|  4 |    40 |    0 |    1 | 2180 |    0 |    2 |    5 |   60 |   32 |   79 |   71 |   95  |
|  5 |     0 |    0 |   23 |    4 |  331 |  199 |  274 |  172 |  173 |   10 |  455 |    0  |
|  6 |     9 |    0 |   20 |    1 |   22 | 1023 |   57 |  183 |   39 |   39 |   76 |    3  |
|  7 |    11 |    1 |   20 |    9 |    0 |   82 |  642 |   68 |   12 |   17 |    9 |   51  |
|  8 |    99 |    0 |   19 |    3 |    3 |  447 |  324 | 1762 |   30 |   20 |  123 |   16  |
|  9 |     1 |    0 |  231 |    6 |    0 |   51 |  154 |   23 |  188 |    0 |  118 |    0  |
| 10 |    10 |    0 |   47 |    6 |    0 |    0 |    0 |   24 |    1 |  145 |    2 |    2  |
| 11 |     0 |    0 |   57 |   87 |    0 |  294 |   95 |  531 |  136 |   24 | 1250 |    0  |
| 12 |    48 |    0 |   10 |   60 |    0 |    4 |    0 |   20 |    7 |   11 |   21 | 7072  |